Topic: Project 2    
Subject: Cleaning Roger Ebert Review Data  
Date: 10/06/2017  
Name: Zach Heick

In [6]:
import pickle
import pandas as pd
import numpy as np

In [7]:
df = pd.read_pickle('roger.pickle')

The blank spaces in the dataframe are empty strings. I need to decide whether I wanted to remove rows or fill in those empty cells with a value.

In [8]:
df.sample(5)

,Title,Year,Star_Score,Genre,Sub-genre,Rating,Runtime
5843,The Clan Of The Cave Bear,1986,1.5,Adventure,Drama,R,98
4375,Tank Girl,1995,2.0,Action,Comedy,R,104
6117,Eddie Macon's Run,1983,2.0,Action,Adventure,PG,94
2043,Shark Tale,2004,2.0,Action,Animation,PG,90
4576,Threesome,1994,3.0,Comedy,Drama,R,93


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7443 entries, 0 to 7442
Data columns (total 7 columns):
Title         7443 non-null object
Year          7443 non-null object
Star_Score    7443 non-null float64
Genre         7443 non-null object
Sub-genre     7443 non-null object
Rating        7443 non-null object
Runtime       7443 non-null object
dtypes: float64(1), object(6)
memory usage: 407.1+ KB


In [10]:
df['Year'].value_counts()

2012    307
2011    285
2009    281
2005    280
2002    267
2003    267
2004    264
2010    256
1999    244
2000    243
1998    240
1997    240
2001    234
1996    229
1995    225
2008    210
1994    196
1993    173
2007    168
1991    162
1992    161
1988    159
2006    158
1987    150
1989    144
1990    142
1986    141
1968    115
1983    111
1971    107
1976    107
1980    106
1984    105
1972     96
1969     96
1973     94
1985     93
1982     90
1975     86
1981     82
1967     72
1979     60
1978     47
1970     47
1977     31
1974     31
2013     19
1966      4
          3
1964      2
1963      2
1960      2
1955      2
1965      1
1962      1
1957      1
1952      1
1951      1
1948      1
1927      1
Name: Year, dtype: int64

There are only three movies that did not list a date, so I'll remove these rows.

In [11]:
df.drop(df.index[df['Year'] == ''], inplace=True)

Star score values look good, no need to clean that column.

In [12]:
df['Star_Score'].value_counts()

3.0    2340
3.5    1248
2.0    1229
2.5     860
4.0     810
1.5     419
1.0     385
0.5      86
0.0      63
Name: Star_Score, dtype: int64

In [13]:
df['Genre'].value_counts()

Drama              1949
Comedy             1915
Action             1591
                    827
Documentary         356
Crime               242
Adventure           190
Horror              134
Animation            87
Foreign              37
Family               21
Mystery              20
Thriller             19
Science Fiction      16
Fantasy              10
Suspense              7
Music                 6
Disaster              3
Romance               3
Western               3
Erotic                2
Indie                 1
History               1
Name: Genre, dtype: int64

In [14]:
df['Sub-genre'].value_counts()

                   1716
Drama              1488
Foreign             751
Adventure           660
Romance             446
Comedy              374
Indie               294
Family              289
Crime               284
Thriller            201
Horror              154
Mystery             137
Fantasy             116
Music                83
History              76
Science Fiction      75
Animation            69
Documentary          48
Sports Film          41
Suspense             39
Musical              21
Western              18
Sport                17
War                  10
Disaster             10
Erotic                6
Holiday               5
Eastern               5
Road Movie            5
Film Noir             1
Short                 1
Name: Sub-genre, dtype: int64

In [15]:
df['Rating'].value_counts()

R               2913
PG-13           1521
PG              1118
NR               918
                 644
G                209
No                31
Unrated           29
NC-17             23
X                 14
Not               10
:                  2
PG-13&#8206;       1
PG-                1
g                  1
PG13               1
TV                 1
R,                 1
.                  1
GP                 1
Name: Rating, dtype: int64

`Genre`, `Sub-genre`, and `Rating` all have empty strings. To make replacing empty strings with other values easier , I use `NaN` as a placeholder.

In [16]:
df.replace(r'^\s*$', np.nan, regex=True, inplace = True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7440 entries, 0 to 7442
Data columns (total 7 columns):
Title         7440 non-null object
Year          7440 non-null int64
Star_Score    7440 non-null float64
Genre         6613 non-null object
Sub-genre     5724 non-null object
Rating        6796 non-null object
Runtime       7034 non-null float64
dtypes: float64(2), int64(1), object(4)
memory usage: 465.0+ KB


Many movies do not have sub-genres which I feel is a valid assumption. Also, movies that do not list a specific genre where assumed to be equivalent to 'NR'.

In [17]:
df['Sub-genre'].fillna('No Sub-genre', inplace=True)
df['Rating'].fillna('NR', inplace=True)

While scraping I got some weird characters for `Rating`. This could have been caused by some inconsistent HTML or labeling. I'll make a table of all movies that have one of these strange ratings and manually search what their rating is on Ebert's review site.

In [18]:
df[
    (df['Rating'] == ':') |
    (df['Rating'] == 'PG-') |
    (df['Rating'] == '.') |
    (df['Rating'] == 'TV') |
    (df['Rating'] == 'g') |
    (df['Rating'] == 'GP')
]

,Title,Year,Star_Score,Genre,Sub-genre,Rating,Runtime
10,Phil Spector,2013,2.5,NaN,No Sub-genre,TV,92.0
3804,Fire,1997,3.0,Drama,Foreign,.,104.0
4063,A Very Brady Sequel,1996,2.5,Comedy,No Sub-genre,g,90.0
5172,Narrow Margin,1990,1.5,Action,Crime,:,97.0
5173,The Tall Guy,1990,3.5,Comedy,Romance,:,90.0
5289,Limit Up,1989,2.0,Comedy,Drama,PG-,89.0
7140,The Revolutionary,1970,4.0,NaN,No Sub-genre,GP,92.0


In [19]:
df['Rating'].replace('No', 'NR', inplace=True)
df['Rating'].replace('Not', 'NR', inplace=True)
df['Rating'].replace(':', 'R', inplace=True)
df['Rating'].replace('TV', 'NR', inplace=True)
df['Rating'].replace('PG-', 'PG-13', inplace=True)
df['Rating'].replace('.', 'NR', inplace=True)
df['Rating'].replace('GP', 'PG', inplace=True)
df['Rating'].replace('g', 'G', inplace=True)
df['Rating'].replace('R,', 'R', inplace=True)
df['Rating'].replace('PG-13&#8206;', 'PG-13', inplace=True)
df['Rating'].replace('PG13', 'PG-13', inplace=True)

The `Rating` column now contains valid MPAA ratings.

In [20]:
df['Rating'].value_counts()

R          2916
NR         1605
PG-13      1524
PG         1119
G           210
Unrated      29
NC-17        23
X            14
Name: Rating, dtype: int64

There were still some movies that did not have a genre, runtime, or both. I could not use these for my model so I drop these rows that were empty in one or both of these columns.

In [18]:
df.dropna(subset=['Genre', 'Runtime'], how='any', inplace=True)

In [36]:
df.reset_index(drop=True, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6509 entries, 0 to 6508
Data columns (total 7 columns):
Title         6509 non-null object
Year          6509 non-null int64
Star_Score    6509 non-null float64
Genre         6509 non-null object
Sub-genre     6509 non-null object
Rating        6509 non-null object
Runtime       6509 non-null float64
dtypes: float64(2), int64(1), object(4)
memory usage: 356.0+ KB


I pickle the cleaned dataframe into a new file. I now have a clean dataframe containing information about Roger Ebert's movie reviews, but I still wanted to explore data from other sources.

In [35]:
df.to_pickle('roger_clean_no_other_data.pickle')